In [ ]:
from selenium import webdriver
import pandas as pd
import numpy as np
import time

# Login to fb @news page
def login(driver, account, password):
    # To login page
    driver.find_element_by_xpath("//*[@id=\"mobile_login_bar\"]/div[2]/a[1]").click()

    # Enter account
    driver.find_element_by_xpath("//*[@id=\"m_login_email\"]").send_keys(account)

    # Enter password
    driver.find_element_by_xpath("//*[@id=\"m_login_password\"]").send_keys(password)

    # Submit login
    driver.find_element_by_xpath("//*[@id=\"u_0_5\"]").click()

# Scroll down to load more articles @base
def scrollDown5times(driver):
    for i in range(5):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
    print('scrolled down 5 times.')

# Get all unique article ids
def getArticleIDsFromBase(driver, last_end):
    try:
        articles = driver.find_elements_by_tag_name('article')
        article_ids = []
        for article in articles[last_end:]:
            tag_info = article.get_attribute("data-store")
            start_idx = tag_info.find('mf_story_key')+len('mf_story_key')+1
            end_idx = tag_info.find(':top_level_post_id')
            article_ids.append(tag_info[start_idx:end_idx])
        print(str(len(article_ids))+' articles found.')
        end = len(articles)
        return article_ids, end
    except:
        raise SystemExit('NO articles found.')

def saveResults(page, article_ids):
    df_ids = pd.DataFrame(np.array(article_ids))
    df_ids.to_csv('results/'+page+'_ids.csv', encoding='utf-8', header=None, index=False)
       
def get(page, base, scrolls, account, password):
    # Start webdriver and nav to base
    # if chromedriver error, find path to chromedriver and pass in
    # executable_path=r"path/to/chromedriver" to webdriver.Chrome()
    driver = webdriver.Chrome()
    driver.get(base)
    
    # Login to fb
    login(driver, account, password)

    # Get article ids while scrolling down every 2 times
    article_ids = []
    last_end = 0
    last_height = driver.execute_script("return document.body.scrollHeight")
    no_increase_counter = 0
    for i in range(scrolls):
        scrollDown5times(driver)

        # Backup break condition: if page not scrolling down
        #new_height = driver.execute_script("return document.body.scrollHeight")
        #if new_height == last_height:
            #saveResults(article_ids)
        #last_height = new_height

        # add to results list
        article_ids_temp, end = getArticleIDsFromBase(driver, last_end)
        article_ids.extend(article_ids_temp)

        # Break loop if total articles have not been increasing for 3 scrolls
        if end == last_end:
            no_increase_counter += 1
            if no_increase_counter == 3:
                saveResults(page, article_ids)
                raise SystemExit("scrolls not registering at %ith scroll." % i)

        # Update total articles count
        last_end = end
        print('Current total articles: %i' % last_end)

        # Show how many scrolls left
        if i%20 == 0:
            left = scrolls - i
            print('%i scrolls left' % left)

    # Save article_ids to csv
    saveResults(page, article_ids)